In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random, os
from glob import glob
import cv2
from PIL import Image

In [2]:
tf.keras.backend.clear_session()

In [3]:
WIDTH = 224 # 저는 VGG16을 참고하여 구현을 하였습니다. 그래서 가로, 세로 224사이즈로 조정하기로 했습니다.
HEIGHT = 224
CHANNEL = 3

In [4]:
os.getcwd()

'/home/ichanho/workspace/Chanho/Pistachio_Image_Dataset/Pistachio_Image_Dataset'

In [5]:
current_path = os.getcwd()

In [6]:
glob(current_path + "/*") # 현재 경로를 생각하여 각각의 이미지에 대한 경로를 확인하기 위해 사용했습니다.

['/home/ichanho/workspace/Chanho/Pistachio_Image_Dataset/Pistachio_Image_Dataset/image_at_epoch_0027.png',
 '/home/ichanho/workspace/Chanho/Pistachio_Image_Dataset/Pistachio_Image_Dataset/image_at_epoch_0022.png',
 '/home/ichanho/workspace/Chanho/Pistachio_Image_Dataset/Pistachio_Image_Dataset/image_at_epoch_0010.png',
 '/home/ichanho/workspace/Chanho/Pistachio_Image_Dataset/Pistachio_Image_Dataset/image_at_epoch_0006.png',
 '/home/ichanho/workspace/Chanho/Pistachio_Image_Dataset/Pistachio_Image_Dataset/image_at_epoch_0018.png',
 '/home/ichanho/workspace/Chanho/Pistachio_Image_Dataset/Pistachio_Image_Dataset/Kirmizi_Pistachio',
 '/home/ichanho/workspace/Chanho/Pistachio_Image_Dataset/Pistachio_Image_Dataset/Deep Dream.ipynb',
 '/home/ichanho/workspace/Chanho/Pistachio_Image_Dataset/Pistachio_Image_Dataset/keras_model',
 '/home/ichanho/workspace/Chanho/Pistachio_Image_Dataset/Pistachio_Image_Dataset/image_at_epoch_0016.png',
 '/home/ichanho/workspace/Chanho/Pistachio_Image_Dataset/Pista

In [7]:
Kirmizi_Pistachi_Path = '/home/ichanho/workspace/Chanho/Pistachio_Image_Dataset/Pistachio_Image_Dataset/Kirmizi_Pistachio/*'
Siirt_Pistachio_Path = '/home/ichanho/workspace/Chanho/Pistachio_Image_Dataset/Pistachio_Image_Dataset/Siirt_Pistachio/*'

In [8]:
kirimi_dataset = glob(Kirmizi_Pistachi_Path) # glob이라는 라이브러리를 활용하여 각각의 파일의 절대 경로를 리스트에 담았습니다.
len(kirimi_dataset) # kirimi 데이터셋의 개수를 1232개 였습니다.

1232

In [9]:
siirt_dataset = glob(Siirt_Pistachio_Path)
len(siirt_dataset) # siirt 데이터셋의 개수는 916개 였습니다.

916

### 두 종류의 데이터셋이 굉장히 적다고 생각하였습니다. 
### 제가 인공지능에 대해 자세히 알지는 못하지만, 하나의 객체를 확인하기 위해서는 N만장의 데이터가 필요한 것으로 알고있었기 때문입니다.
### dogs-vs-cats classification에 관한 공부를 할 때도, 몇만장이 있었던 것으로 기억합니다.

##### 사진 파일 이외에 다른 파일이 있는 찾아보는 과정입니다

In [10]:
for i in kirimi_dataset:
    if(i[-3:] != 'jpg'):
        print(i)

In [11]:
for i in siirt_dataset:
    if(i[-3:] != 'jpg'):
        print(i)

#### 다른 파일이 있으면 무엇이라도 출력이되어야 하는데, 아무것도 출력되지 않습니다.
#### 그래서 바로 데이터 전처리에 시행했습니다.
#### 앞서 말했듯이 600x600의 이미지를 244x244로 다운 스케일링하고, 채널은 유지했습니다.
#### 이미지를 보다 작은 사ㅣ즈로 보간하는 것이기에 Image.LANCZOS 보간법을 사용하여 최대한 원본 이미지를 유지하였습니다.

In [12]:
datasets = []
answers = []
for i in kirimi_dataset:
    kirimi_input = Image.open(i)
    kirimi_input.convert('RGB')
    kirimi_input = kirimi_input.resize((HEIGHT, WIDTH), Image.LANCZOS)
    # 이미지를 다운 스케일링할 때 왜곡이 발생하게 되는데, 작은 이미지에 대한 보간법으로 Image.LANCZOS가 가장 적합다는 Pillow Document를 참조하여 사용했습니다.
    datasets.append(np.array(kirimi_input))
    answers.append(np.array([0, 1])) # 원 핫 인코딩을 하는 과정입니다.

for i in siirt_dataset:
    siirt_input = Image.open(i)
    siirt_input.convert('RGB')
    siirt_input = siirt_input.resize((HEIGHT, WIDTH), Image.LANCZOS)
    datasets.append(np.array(siirt_input))
    answers.append(np.array([1, 0]))
    
    
print(len(datasets), len(answers))

/home/ichanho/.local/share/virtualenvs/Chanho-Ro_Xhmwc/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  
/home/ichanho/.local/share/virtualenvs/Chanho-Ro_Xhmwc/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  


2148 2148


#### Tensorflow 2.6 의 Sequential을 이용하여 모델을 만들고
#### model fit을 하기 위해 numpy 데이터로 형변환 합니다.

In [13]:
test = np.array(datasets)
valid = np.array(answers)

In [14]:
print(test.shape)
print(valid.shape)

(2148, 224, 224, 3)
(2148, 2)


In [15]:
type(test[0])

numpy.ndarray

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

### VGG16 을 기반하여 모델을 구현하였습니다.
### 그림을 참조하여 스스로 구현하였습니다.₩

In [17]:
model = Sequential()

2022-05-25 22:30:14.035108: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-25 22:30:14.054114: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-25 22:30:14.054229: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-25 22:30:14.054560: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [18]:
model.add(Conv2D(64, (3,3), activation="relu", padding='same',input_shape=(HEIGHT, WIDTH , CHANNEL)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3,3), activation="relu", padding='same'))
model.add(Conv2D(128, (3,3), activation="relu", padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256, (3,3), activation="relu", padding='same'))
model.add(Conv2D(512, (3,3), activation="relu", padding='same'))
model.add(Conv2D(256, (3,3), activation="relu", padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(512, (3,3), activation="relu", padding='same'))
model.add(Conv2D(512, (3,3), activation="relu", padding='same'))
model.add(Conv2D(512, (3,3), activation="relu", padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(512, (3,3), activation="relu", padding='same'))
model.add(Conv2D(512, (3,3), activation="relu", padding='same'))
model.add(Conv2D(512, (3,3), activation="relu", padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
# model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation='relu'))
# model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(2,activation="softmax")) # sigmoid

### 이번 분류는  A or B 처럼 두 종류의 데이터 타입을 분류하는 것이기에 binary_crossentrophy를 사용하였습니다.ㅏ
### optimizer는 RMSprop을 사용하였으며, 학습율은 0.0001로 하였습니다.

In [19]:
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4), metrics=['accuracy']) # binary_...
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 56, 56, 256)       2

### 여기는 교수님께서 요청하셨던 데이터의 비율을 7:2:1로 나누는 작업니다.

In [20]:
x_train, x_test, y_train, y_test = train_test_split(test, valid, test_size=0.3, shuffle=True, random_state=34)

In [21]:
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.3, shuffle=True, random_state=34)

In [22]:
cp_path = './keras_model/model.cp'

callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, verbose=1),
    tf.keras.callbacks.ModelCheckpoint(cp_path, save_best_only=True, save_weights_only=True)
]
model.fit(x_train, y_train, epochs=50, validation_data=(x_val, y_val), callbacks=callbacks)

Epoch 1/50


2022-05-25 22:30:16.012138: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8303
2022-05-25 22:30:18.350319: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


47/47 [==============================] - 23s 370ms/step - loss: 0.8064 - accuracy: 0.7112 - val_loss: 0.4247 - val_accuracy: 0.8093
Epoch 2/50
47/47 [==============================] - 13s 267ms/step - loss: 0.4632 - accuracy: 0.7984 - val_loss: 0.4584 - val_accuracy: 0.7605
Epoch 3/50
47/47 [==============================] - 13s 267ms/step - loss: 0.4623 - accuracy: 0.8017 - val_loss: 0.4505 - val_accuracy: 0.8004
Epoch 4/50
47/47 [==============================] - 13s 273ms/step - loss: 0.4113 - accuracy: 0.8217 - val_loss: 0.3674 - val_accuracy: 0.8226
Epoch 5/50
47/47 [==============================] - 13s 267ms/step - loss: 0.3940 - accuracy: 0.8283 - val_loss: 0.4103 - val_accuracy: 0.8115
Epoch 6/50
47/47 [==============================] - 13s 267ms/step - loss: 0.3586 - accuracy: 0.8417 - val_loss: 0.3989 - val_accuracy: 0.8293
Epoch 7/50
47/47 [==============================] - 13s 268ms/step - loss: 0.3396 - accuracy: 0.8589 - val_loss: 0.3773 - val_accuracy: 0.8182
Epoch 8/50

In [23]:
evaluate_model = model.evaluate(x_test, y_test)

7/7 [==============================] - 1s 145ms/step - loss: 0.4097 - accuracy: 0.8814


In [24]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [25]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

(1503, 224, 224, 3)
(1503, 2)
(451, 224, 224, 3)
(451, 2)
(194, 224, 224, 3)
(194, 2)


In [26]:
pred = model.predict(x_test, verbose=0)
print(np.max(pred) * 100)

100.0


In [27]:
preprocess_pred = []
for i in pred:
    if(i[0] > i[1]):
        preprocess_pred.append(1)
    else:
        preprocess_pred.append(2)


In [28]:
preprocess_y_test = []
for i in y_test:
    if(i[0] > i[1]):
        preprocess_y_test.append(1)
    else:
        preprocess_y_test.append(2)


In [29]:
precision = precision_score(preprocess_y_test, preprocess_pred)
recall = recall_score(preprocess_y_test, preprocess_pred)
f1_scores = f1_score(preprocess_y_test, preprocess_pred)
 
print('Precision: ',precision)
print('Recall: ',recall)
print('F1-score :  ', f1_scores)

Precision:  0.8625
Recall:  0.8518518518518519
F1-score :   0.8571428571428572
